In [82]:
import numpy as np
import pandas as pd


In [83]:
df = pd.read_csv(r"C:\Users\Lenovo\Desktop\projet_baina\nouveau_projet\final_code\data\transfert_data.csv")

In [84]:

print("Shape of df:", df.shape)
df1 = df.copy()


Shape of df: (798, 20)


In [85]:
df1=df1.drop_duplicates()
df1=df1.reset_index(drop=True)
df1.shape

(798, 20)

In [86]:
df1=df1.drop(["Unnamed: 0","country_from","country_to"],axis=1)

In [87]:
occurrences_clubs = df1["club_from"].value_counts().unique()
occurrences_clubs

array([15, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1], dtype=int64)

In [88]:
# Colonnes catégorielles à traiter
colonnes_categorielles1 = ['league_from', 'league_to']
colonnes_categorielles2 = ['club_to', 'club_from']

# Traitement des colonnes categorielles
for colonne in colonnes_categorielles1:
    counts = df1[colonne].value_counts()
    categories_autre = counts[counts < 4].index.tolist()
    df1[colonne] = df1[colonne].apply(lambda x: 'Autre' if x in categories_autre else x)

for colonne in colonnes_categorielles2:
    counts = df1[colonne].value_counts()
    categories_autre = counts[counts < 11].index.tolist()
    if 'Saudi Pro League' in categories_autre:
        categories_autre.remove('Saudi Pro League')
    df1[colonne] = df1[colonne].apply(lambda x: 'Autre' if x in categories_autre else x)

# One-hot encoding
df_encoded = pd.get_dummies(df1, columns=colonnes_categorielles1 + colonnes_categorielles2 + ['position'] + ['loan'], dtype=int)


In [89]:
df_encoded["fee"] = df_encoded["fee"].str.replace('k', '').astype(float) * 1000
df_encoded["market_value"] = df_encoded["market_value"].str.replace('k', '').astype(float) * 1000


In [90]:
df_encoded=df_encoded.dropna()
df_encoded=df_encoded.drop_duplicates()

In [91]:
df_encoded.iloc[:,4:11].corr().abs()

,ATT,SKI,MOV,POW,MEN,DEF,GK
ATT,1.000000,0.906219,0.653273,0.827636,0.840280,0.157337,0.783897
SKI,0.906219,1.000000,0.699717,0.714497,0.872538,0.285973,0.753965
MOV,0.653273,0.699717,1.000000,0.507865,0.559894,0.076811,0.563624
POW,0.827636,0.714497,0.507865,1.000000,0.781490,0.206922,0.603301
MEN,0.840280,0.872538,0.559894,0.781490,1.000000,0.528228,0.727101
DEF,0.157337,0.285973,0.076811,0.206922,0.528228,1.000000,0.444797
GK,0.783897,0.753965,0.563624,0.603301,0.727101,0.444797,1.000000


In [92]:
df_encoded=df_encoded.reset_index(drop=True)

In [93]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.utils import shuffle
n_rows_to_generate = 500000 - len(df_encoded)
X = df_encoded.drop(columns=["fee"])
y = df_encoded["fee"]
X_synthetic, y_synthetic = make_classification(
    n_samples=n_rows_to_generate,
    n_features=X.shape[1],
    n_classes=len(np.unique(y)),
    weights=[0.5] * len(np.unique(y)),  
    random_state=42
)
df_synthetic = pd.DataFrame(data=X_synthetic, columns=X.columns)
df_synthetic["fee"] = y_synthetic
df_final = pd.concat([df_encoded, df_synthetic], ignore_index=True)
df_final.to_csv(r"C:\Users\Lenovo\Desktop\projet_baina\nouveau_projet\final_code\data\model_training_data.csv", index=False)


ValueError: n_classes(132) * n_clusters_per_class(2) must be smaller or equal 2**n_informative(2)=4